### Want to try the WFA with IBIS data

** are for ideas that might be tested before, in the spectroscopy exercise

The data is in the Google drive, IBIS_polarimetry folder https://drive.google.com/drive/folders/1E9j7ykrUbeVRw3XLfYHgPGKLhF_7BnZg?usp=sharing

#### read in the data. Nice (or just causes more trouble?), we have both CaII 8542 and FeI 6302, simultaneous (within the sampling time, probably around 1 min)

Ca [217,419,35,4]   # x, y, lambda, S [I, Q, U, V]
Fe [217,419,23,4]

pixel size =0.082", the two FOVs do overlap. I think destretching has been applied, but no reconstruction

same for the wavelengths

ca_wav[35]
fe_wav[23]

all has been calibrated: average core wavelength within quiet area of Sun = nominal wavelength;
intensity calibrated to continuum = 10,000 using the atlas as a comparson 

The Stokes profiles are calibrated to the best of my knowledge, but there could be some residual cross-talk

? Exposure times - ask Kevin

#### Plot some average / quiet pixel profiles. Note that there is no real continuum in the CaII data, but in the FeI, yes (p.s. the turn towards lower values in the red wing of the FeI is because there is a telluric line nearby!)

** If we want to show the atlas, it might bring home the concept of "convolution / spectral resolution" (maybe do it in the spectroscopy exercise? **

Also, the wavelengths are not sampled evenly. The FeI yes, apart from the continuum point. The CaII instead has more points in the core. This will become important when calculating the WFA - as one needs to calculate the derivative of the profile, we need to pay attention to the wavelength steps.

From now we concentrate on the CaII 


#### In the previous exercise (and with FISS) we have stepped through the wavelengths looking at how the intensity changes, and noticed how different parts of  the line sample different parts of the atmosphere. Let's do it again quickly to get a sense of the FOV. 


considerations: the wings of CaII show very clearly the magnetic elements; a lot of "fibrils" appear at ~ 0.4 Ang
from core; the core of Fe gets just above the "layer" of reversed granulation, etc


#### What happens when we do the same but using Stokes Q, U, V ? 

.... 

Note how the maximum / minimum of the signal happens at different wavelength vs. the intensity, and how it changes sign on either side of the line core. The imagery brings it home very well. An interesting fact is that in the extreme wings of CaII, even among a lot of noise, the V signal is very similar to FeI, while as you move towards the core of CaII, the pattern changes (more diffuse, at least)

Q, U movies show the "rotation" of signs at different wavelengths! (rotation because we have a circular spot)
Q, U are obviously more noisy than V


#### We can then plot some Stokes profiles in interesting pixels (spot, little pores, quiet) and see how they look

E.g. quiet: [77,57], near spot: [68,225]; pore: [171,161]; plage [207,179] (with x-axis = the shorter dimension, 217)

Some of the profiles are very weird; will the WFA work for them? we'll see / discuss

#### Go straight to the WFA ? We have all the info we need, and have seen from both Rebecca and Brian how to do it ...

Two maybe important differences: 1. we don't have much spectral coverage in the wings; and 2. the spectral points are not sampled evenly.

Discuss?

[my answers: 1. the WFA works well when restricted to the core of the line, so we should be safe there; we won't be able to try to get the photospheric field from the wing like Brian showed, but oh well; 2. this might have an effect on the numerical precision of our calculation? Not so sure .... the part where we'd apply the WFA is sampled (almost) evenly at 38 mA ... and any derivative function should be able to take into account a varying x-axis

Maybe we can try to interpolate, or not, and see if the results change?



#### these are the wavelength intervals that Brian uses:

corepos = [ 0.00, 0.25 ]    # in Angstroms, from nominal line-center
cwbypos = [ 0.10, 0.40 ]
wingpos = [ 0.75, 1.25 ]

Then the intervals are used like so: 
LOS: core
TRN: cwby
(PHOT: wing)

#### So we know the formula, right:

V = - C_long dI/dlambda   with
C_long = B_los * g_eff * lambda^2 * 4.67 E-13   (lambda in Ang. = 8542.09; g_eff (B_los) = 1.1; B_los in Gauss)

meaning, there is (must be) a linear relationship between the V signal and the dI/dlambda, mediated by B
And we know that this is valid in a certain wav. range
    

#### So let's pick a pixel where we expect magnetic signal, plot the V profile, and plot -dI/dl (2 panels). 
E.g. near the umbra: (68,225)
[spare: or the plage just below: (135,149)]


Do they look similar? do we need to restrict the wav. range to see similarities?

Let's try the linear fit (remember to divide U, Q, V by 10,000, if you want a fractional polarization), and see what numbers the little code spits out

(hopefully turns out allright!)

Maybe interpolating before doing the derivative would have been better? TBD

#### Let's repeat the exercise with the linear polarization. 

Now the formula is different: 

L = sqrt(U^2+V^2) = 3/4 (B_tran)^2 * abs(C_tran* 1/(lambda-lambda0)) * abs(dI/dlambda)
VALID ONLY IN THE WINGS!  (note, Rebecca uses the far wings: from 2 Ang away fromline core, to 5,6 ,7 .....
Brian uses something like 0.1

or, sometimes:

L = sqrt(U^2+V^2) = 1/4 * (B_tran)^2 * C_tran * d^2I/d^2lambda
VALID ONLY N THE VERY CORE

For both: 
with C_tran  = Geff* (4.6686e-13 * lam0^2)^2 # for lambda in Angs, field in G

lam0 = 8542.09    #in Angs
Geff = 1.21   # Lande factor for Btrans (Brian uses 1.205)


Like before, let's plot the two quantities side by side (not on top of each oher as they have different amplitudes, mediated by B) and do our considerations





We will first use the equation that relates the linear polarization to the first derivative of Stokes I, which is typically valid in the wings of the line:

$L(\lambda_0) = \frac{3}{4}B_{T}^2 \left|C_{T}\frac{1}{\lambda_w-\lambda_0}\right|\left|\frac{\partial I}{\partial \lambda}\right|$

#### Inclination would then be 
    binc = np.arctan2( btrn, blos )* (180./np.pi)

#### and azimuth
az = 0.5* np.arctan2( np.sum( data[2,:] ), np.sum( data[1,:] ) ) * (180./np.pi) + 90. ; using Q, U

#### OK, how about doing it for the whole map? 

We can interpolate pixel by pixel, apply the formulae above and/or use the little module of Brian's, and do the full FOV. 

***** CAREFUL : BRIAN'S DATA HAS DIMENSIONS (S, NLAMBDA), SO WE HAVE TO BE CAREFUL ON HOW WE FEED THE ALGORITHM



This they can do by themselves? Including plotting the maps, take a look at the histograms, evaluate the noise? 

They could also rebin the data 2x2 (np.squeeze?) to keill the noise some, and then re-run 
Will the results be different? 

There are many pixels that have a value of B way above what we'd expect from a WFA (e.g. 1500 G or more). Why?

Let's check some of those pixels, like Brian did. Either the noise is too high, or the profiles are very strange, and derivatives have a very peculiar shape. Maybe we can look at how good is the (linear) fit and use that as a criterion to decide which field to believe, or not (I imagine that the linfit of numpy has a way to preserve the goodness of the fit?)


#### Finally, I would try to run , quickly the same thing for FeI 6302. 

WFA is not very viable in most of the field, but the important thing will be to show the different spatial distribution of the maps: sharper and with many details for FeI, much more diffuse (and lower values) for Fe.

???? Would it be worthy to reserrect the py_milne of last year ???